<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import sklearn
import numpy as np
import pandas as pd

In [95]:
from google.colab import drive
drive.mount('/content/drive')
wildfires_larger = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfires1_with_snow.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now that we have gathered data from kaggle, NOAA, and the California Department of Water resources into two consistent datasets, we will drop columns that are not necessary for training our models and take care of some final preprocessing.

In this file, I will 

In [96]:
print(wildfires_larger.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens'],
      dtype='object')


In [83]:
print(wildfires_recent.columns)

NameError: ignored

Drop all columns that will not be relevant for our classification task:
Name, AcresBurned, Cause, SOURCE_REPORTING UNIT NAME, DaysBurn, Discovered DOY, Contained Month, Contained DOY, Lat, Long, County Ids, State, OWNER_DESCR, NOAA Station, Link, Snow Station, River Basin

In [97]:
wildfires_larger.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'Cause',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month', 'Contained Month',
                      'Contained DOY','Latitude','Longitude','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin'], inplace = True)

In [ ]:
wildfires_recent.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Cause',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month', 'Contained Month',
                      'Contained DOY','Latitude','Longitude','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin'], inplace = True)

Double check our datatypes before we proceed with preprocessing and model building. As we can see, all features besides county and fire size rank are numerical (of type float) so all we have to do is one hot encode these two categorial 

In [98]:
wildfires_larger.head()

,AcresBurned,Fire Size Rank,Discovered DOY,County,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens
0,0.10,A,33.0,Plumas,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,49.06,0.0,19.0,1.0,79.6,34.0,0.43
1,0.25,A,133.0,Placer,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,14.76,0.0,3.0,1.0,108.6,38.1,0.35
2,0.10,A,152.0,El Dorado,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,40.37,0.0,11.0,1.0,108.6,38.1,0.35
3,0.10,A,180.0,Alpine,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33
4,0.10,A,180.0,Alpine,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33


In [99]:
print(wildfires_larger['Fire Size Rank'].value_counts())

A    38308
B    18930
C     2684
D      657
E      432
F      327
G      234
Name: Fire Size Rank, dtype: int64


In [100]:
w = wildfires_larger.loc[wildfires_larger['AcresBurned'] >= 3]
print(w['Fire Size Rank'].value_counts())

B    3540
C    2684
D     657
E     432
F     327
G     234
Name: Fire Size Rank, dtype: int64


In [110]:
w['New Fire Size Rank'] = ''
for index, rows in w.iterrows():
  size_fire = w.at[index, 'AcresBurned']
  if size_fire >= 3 and size_fire <= 5:
    size_class = 'A'
  elif size_fire > 5 and size_fire <= 40:
    size_class = 'B'
  elif size_fire > 40:
    size_class = 'C'
  w.at[index,'New Fire Size Rank'] = size_class

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
print(w['New Fire Size Rank'].value_counts())

B    3069
A    2466
C    2339
Name: New Fire Size Rank, dtype: int64


In [ ]:
wildfires_larger.info()

In [ ]:
wildfires_larger.describe()

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

Address Null Values and Data Smoothing

One-hot encode categorical features (e.g. Month discovered)

Bar Charts and Class Definition

Split the data into train-test sets

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

Normalize data using min-max-scalar after splitting into train and test

Feature Selection

In [ ]:
from sklearn.feature_selection import RFE, SelectKBest

Cross-Validation

Model Building:

SVM Classifier

In [ ]:
from sklearn import svm
from sklearn.svm import SVC

KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

Decision Tree Classifier

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Gradient Boost Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

Neural Network Multiclass classifier (TensorFlow)
- Experiment by changing number of hidden layers and activation functions (sigmoid, relu, softmax)
- Change number of epochs and add more hidden layers
- Size of input = number of features in the dataset
- Size of output = number of classes in the multiclass classification problem

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier